Group Members
- Jonathan Conn
- Matt Boulden
- Arunam Gupta


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
dow_df = pd.read_csv('data/dow_data.csv')
nas_df = pd.read_csv('data/nasdaq_data.csv')
sp_df = pd.read_csv('data/s&p_data.csv')

dow_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-12-16,28191.669922,28337.490234,28191.669922,28235.890625,28235.890625,286770000
1,2019-12-17,28221.750000,28328.630859,28220.560547,28267.160156,28267.160156,286770000
2,2019-12-18,28291.439453,28323.250000,28239.279297,28239.279297,28239.279297,289890000
3,2019-12-19,28278.310547,28381.480469,28278.240234,28376.960938,28376.960938,262570000
4,2019-12-20,28608.640625,28608.640625,28445.599609,28455.089844,28455.089844,603780000
...,...,...,...,...,...,...,...
248,2020-12-09,30229.810547,30319.699219,29951.849609,30068.810547,30068.810547,380520000
249,2020-12-10,30032.550781,30063.869141,29876.820313,29999.259766,29999.259766,325550000
250,2020-12-11,29988.210938,30071.130859,29820.839844,30046.369141,30046.369141,393870000
251,2020-12-14,30123.910156,30325.789063,29849.150391,29861.550781,29861.550781,371980000
